# **Histograms of Oriented Gradients**

### **Import**

In [26]:
import cv2
import numpy as np
import tqdm as t
import sklearn.neighbors as sn
import pandas as pd

### **Histograms of Oriented Gradients Parameter**

In [27]:
featureTr = [];# สร้าง array ไว้เก็บ Feature เเต่ละรูปภาพ
labelTr = [];# สร้าง array ไว้เก็บ Feature เเต่ละรูปภาพ

In [28]:
# HOG' s Parameter
winSize = (64,64) #ปรับขนาดของ Window
blockSize = (16,16) #ปรับขนาดของบล็อก
blockStride = (4,4) #ปรับขนาดของการขยับของบล็อก
cellSize = (8,8) #กำหนดขนาด cell
nbins = 9#จำนวนบินใน Histogram
derivAperture = 1
winSigma = 4.0 #ปรับ Window ของ Gaussian smoothing
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01 #ปรับการหดตัวของ L2-Hys
gammaCorrection = 0 #ปรับว่าต้องประมวลผลก่อนการแก้ไขแกมมาหรือไม่
nlevels = 64 #ปรับการ Quantization
winStride = (4,4) #ปรับขนาดของการขยับของ Window
padding = (4,4) #ปรับการเติมขอบของภาพ
locations = ((10,20),) #จุดของ vector

In [29]:
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma, # นำค่า parameter ที่เก็บค่าใน  HOG (ตัวสกัดคุณลักษณะ)
histogramNormType,L2HysThreshold,gammaCorrection,nlevels)


### **HOG Training**

In [30]:
# Training Image Loader and Feature Extraction
for _classname in t.tqdm(range(1,7)): # loop จำนวน class 1-6
  for _id in range(1,21): # loop เเต่ละรูปภาพ 20 รูป
    path = '/content/drive/MyDrive/ImgRecPJ2/Tr/' + str(_classname) + '/text (' + str(_id) + ').bmp' #ตัวอย่าง path ที่เก็บ= '/content/drive/MyDrive/Tr/6/text (20).bmp'

    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) #ปรับรูปภาพให้อยู่ใน Gray scale เพื่อให้สามารถทำ Radon ได้
    img = np.invert(img) #ปรับสีจากขาวเป็นดำสลับกัน
    img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA) #ปรับขนาดภาพให้เป็น 128 x 128

    # print(img.shape)

    #ทำ Opening เพื่อย่อเเละเพิ่มขนาดของวัตถุหรือเส้นขอบให้สกัดรูปภาพ(ตัวอักษรหนา)ได้ดีขึ้น
    filt = np.ones((2,2)) # กำหนดขนาด filter ใช้ทำ Opening
    #img  = cv2.dilate(img, filt, iterations = 1) #ทำ Dialtion 1 รอบ
    img  = cv2.morphologyEx(img, cv2.MORPH_CLOSE, filt) #ทำ Closing Operation
    
    hist = hog.compute(img,winStride,padding,locations) # ทำสกัด Feature ของรูปภาพ

    # print(hist)
    # print(np.array(hist).shape) # เเสดงลักษณะข้อมูลหลังกัดรูปภาพ
    featureTr.append(np.array(hist)) # เก็บข้อมูล Feature ใน array ชื่อ featureTr
    labelTr.append(_classname) # เก็บข้อมูลชื่อ class ใน array ชื่อ labelTr

featureTr = np.array(featureTr).reshape(120,-1) #เป็นเเนวข้อมูลให้เป็นเเนวนอนเพื่อให้ข้อมูลจำเเนกภาพของเครื่องมือ KNN

100%|██████████| 6/6 [00:00<00:00, 13.98it/s]


### **HOG Testing**

In [37]:
# Testing Image Loader and Feature Extraction
# path = '/content/drive/MyDrive/ImgRecPJ2/Tr/6/text (20).bmp'; # ที่วาง path  ไฟล์ที่จะวางทดสอบ
path = '/content/drive/MyDrive/ImgRecPJ2/ซ3.png'; # ที่วาง path  ไฟล์ที่จะวางทดสอบ
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) #ปรับรูปภาพให้อยู่ใน Gray scale เพื่อให้สามารถทำ Radon ได้
img = np.invert(img) #ปรับสีจากขาวเป็นดำสลับกัน
img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA) #ปรับขนาดภาพให้เป็น 128 x 128

# print(img.shape)

#ทำ Opening เพื่อย่อเเละเพิ่มขนาดของวัตถุหรือเส้นขอบให้สกัดรูปภาพ(ตัวอักษรหนา)ได้ดีขึ้น
filt = np.ones((2,2)) # กำหนดขนาด filter ใช้ทำ Opening
#img  = cv2.dilate(img, filt, iterations = 1) #ทำ Dialtion 1 รอบ
img  = cv2.morphologyEx(img, cv2.MORPH_CLOSE, filt) #ทำ Closing Operation
    
hist = hog.compute(img,winStride,padding,locations) # ทำสกัด Feature ของรูปภาพ

labelTs = 6 #กำหนด จำนวน class ที่เอาไป train
classifier = sn.KNeighborsClassifier(n_neighbors=1) # กำหนดค่าหาเพื่อนบ้านที่ใกล์ที่สุด k ตัว ของ ที่ใช้ classifierข้อมูล
classifier.fit(featureTr, labelTr) # เรียนรู้ข้อมูลเเล้วสร้างโมเดลใช้ทำนายค่า
out = classifier.predict(featureTs) # ทำนายข้อมูลวางอยู่
print('Answer is ' + str(out)) #แสดงคำตอบของการ Classify

Answer is [6]


### **Accuracy & Scores**

In [32]:
from sklearn.model_selection import train_test_split # import  sklearn.model_selection เพื่อเเบ่งข้อมูล
x_train, x_test, y_train, y_test = train_test_split(featureTr,labelTr,test_size=0.30,random_state=42) 
#หลังจาก การสกัด Feature ของรูปภาพ ทำการเเบ่งข้อมูล test  30% , train 70 %
# Importing and fitting KNN classifier for k=7
from sklearn.neighbors import KNeighborsClassifier # Import KNN classifier
knn = KNeighborsClassifier(n_neighbors=1) # กำหนดค่าหาเพื่อนบ้านที่ใกล์ที่สุด  7 ตัว หรือ k = 7
knn.fit(x_train,y_train) # เรียนรู้ข้อมูลเเล้วสร้างโมเดลใช้ทำนายค่า
# Predicting results using Test data set
pred = knn.predict(x_test) # ทำนายกลุ่มข้อมูล
from sklearn.metrics import accuracy_score # import sklearn.metrics เพื่อใช้ทำนายความเเม่นยำของโมเดล
accuracy_score(pred,y_test) #ทดสอบเเล้วเเสดง ผลการทดสอบความเเม่นยำของโมเดล

0.8333333333333334

In [33]:
# ทดสอบค่า k ที่ใช้ ทำ classifier(K-Nearest Neighbors) ข้อมูล
knn_r_acc = [] # สร้าง array เก็บผลการทดลองค่าความเเม่นยำของ การเลือกใช้ จำนวน k 1-17 ตัว
for i in range(1,20,1): # loop ทดลองค่าความเเม่นยำของ การเลือกใช้ จำนวน k 1-17 ตัว
    knn = sn.KNeighborsRegressor(n_neighbors=i) # กำหนด k ที่ใช้ทดเเต่ละครั้ง
    knn.fit(x_train,y_train) # เรียนรู้ข้อมูลเเล้วสร้างโมเดลเเต่ละรอบ
    test_score = knn.score(x_test,y_test) # ทำนายเเละเก็บค่า score ของ test เเต่ละรอบ
    train_score = knn.score(x_train, y_train) #ทำนายเเละเก็บค่า score ของ train เเต่ละรอบ
    knn_r_acc.append((i, test_score ,train_score)) # เก็บ ข้อมูล ผลการทดลอง score  ใน array ชื่อ knn_r_acc
df = pd.DataFrame(knn_r_acc, columns=['K','Test Score','Train Score']) 
# นำข้อมูลที่เก็บผลการทดลอง ทำเป็น dataframe
print(df) # เเสดงผลการทดสอบ

     K  Test Score  Train Score
0    1    0.621158     1.000000
1    2    0.765716     0.863290
2    3    0.774024     0.846313
3    4    0.755746     0.806747
4    5    0.747173     0.807481
5    6    0.718637     0.739425
6    7    0.649845     0.700240
7    8    0.648574     0.682351
8    9    0.647743     0.637824
9   10    0.647876     0.603381
10  11    0.589683     0.567540
11  12    0.573041     0.543296
12  13    0.543230     0.509763
13  14    0.508797     0.462434
14  15    0.503428     0.427410
15  16    0.501056     0.414351
16  17    0.478790     0.397927
17  18    0.456937     0.378453
18  19    0.446125     0.343785



The score function is simply a utility function for a default metric to be used within some algorithms of scikit-learn (mostly the algorithms in the model selection module, e.g. GridSearchCV, or cross_validate), if no other metric is specified. So for classification, this is typically accuracy and for regression mean squared error.

So it is the same, as it does exactly what you do in your code: it takes the passed matrix X (e.g. X_test in your case), calls predict and calls accuracy_score. Hence, no surprise that it is the same score. In fact, as scikit-learn is open source, you can just check that yourself here.

### **Play ground**

In [34]:
#  path = '/content/drive/MyDrive/Tr/' + str(_classname) + '/text (' + str(_id) + ').bmp';
#  img = cv2.imread(path)
#  path

In [35]:
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np
# path = '/content/drive/MyDrive/Tr/6/text (15).bmp';
# img = cv2.imread(path)
# filt = np.ones((2,2))
# #img  = cv2.dilate(img, filt, iterations = 1)
# #img  = cv2.erode(img, filt, iterations = 1)
# #img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, filt)
# img = cv2.morphologyEx(img, cv2.MORPH_OPEN, filt)
# img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)
# plt.figure()
# plt.subplot(3,1,1), plt.imshow(img)
# plt.show()